# DCA Strategies Program

## Synopsis
* This program is to see what DCA (dollar cost averaging) investing frequency yields the greatest return on various stocks.

## How to use this program
* Remove the "#" in front of python```!pip install yfinance``` in the next cell and begin running the cells
* in order to generate the data for a specific stock, define a variable using the python```getMaxData() function```
* Enter the exact ticker name in quotations into the python```getMaxData()``` function (search up on yahoo finance if needed)
* To analyze the data, call the python```DCA()``` function. Enter the data from the previously defined variable. There are also options to define parameters for the interval and investment.
* The interval parameter has options for "D" for daily investment,  "W" for weekly investments, "2W" for biweekly investments, and "M" for monthly investments.
* The investment parameter is the total annual investment you would like to make.

- Please note that there are exactly 12 equal monthly investments, 26 equal biweekly investments, 52 equal weekly investments, and 252 equal daily investments as the stock market has 252 trading days.
- By default, the parameters for the DCA function are "D" and 24000 annual investment. 

In [1]:
#!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
import multiprocessing
from datetime import datetime, timedelta

In [3]:
def refactor_table(array):
    """
    this function turns the dataframe table into a 2D array with the date and price
    """
    for i in range(len(array)):
        date_str = str(array[i][0])[:10]  # Format date as 'YYYY-MM-DD'
        price = array[i][1]
        array[i] = [date_str, price]      # Replace entire row
    return array

In [4]:
def getMaxData(ticker):
    """
    this function downloads the ticker data from yahoo finance. You can adjust the period to specific timeframes as well. 
    """
    ticker_dataframe = yf.download(ticker, period="max")['Close']
    ticker_array = ticker_dataframe.reset_index().to_numpy()
    return refactor_table(ticker_array)


In [5]:
def get_interval_prices(ticker, interval):
    """
    this function returns a list of the prices for weekly and biweekly intervals. 
    """
    prices = []
    last_date = None

    for date_str, price in ticker:
        current_date = datetime.strptime(date_str, "%Y-%m-%d")

        if last_date is None or (current_date - last_date).days >= interval:
            prices.append(price)
            last_date = current_date
    return prices

In [6]:
def DCA(ticker, interval='D', investment=24000):
    """
    interval options: 'D', 'W', '2W', 'M'
    """
    results = {}
    prices = []
    stock_count = 0

    # generates the prices table and defines interval investment variables
    if interval == 'D':
        for i in range(len(ticker)):
            prices.append(ticker[i][1])
        interval_investment = investment / 252
    elif interval == 'W':
        for i in range(0, len(ticker), 7):
            prices = get_interval_prices(ticker, 7)
        interval_investment = investment / 52
    elif interval == '2W':
        for i in range(0, len(ticker), 14):
            prices = get_interval_prices(ticker, 14)
        interval_investment = investment / 26
    else:
        seen_months = []
        for i in range(len(ticker)):
            year_month = ticker[i][0][0:7]

            if year_month not in seen_months:
                prices.append(ticker[i][1])
                seen_months.append(year_month)
        interval_investment = investment / 12

    # calculates returns and portfolio value
    for i in range(len(prices)):
        stock_count += interval_investment / prices[i]

    portfolio_value = stock_count * prices[-1]
    portfolio_return = (portfolio_value / investment - 1) * 100

    # returns results
    results =  {
        "annual investment": investment, 
        "interval": interval,
        "interval investment": f"${round(interval_investment, 2)}",
        "portfolio value": f"${round(portfolio_value, 2)}", 
        "portfolio return": f"{round(portfolio_return, 2)}%", 
    }

    for k, v in results.items():
        print(f"{k}: {v}")
    

In [7]:
vfv = getMaxData("VFV.TO")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['VFV.TO']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [8]:
DCA(vfv, 'D')

IndexError: list index out of range

In [ ]:
nvda = getMaxData("NVDA")

[*********************100%***********************]  1 of 1 completed


In [ ]:
DCA(nvda, '2W')

annual investment: 24000
interval: 2W
interval investment: $923.08
portfolio value: $338874953.64
portfolio return: 1411878.97%


In [ ]:
spy = getMaxData("SPY")

[*********************100%***********************]  1 of 1 completed


In [ ]:
DCA(spy, 'D')
print("--------------------------------")
DCA(spy, 'W')
print("--------------------------------")
DCA(spy, '2W')
print("--------------------------------")
DCA(spy, 'M')


annual investment: 24000
interval: D
interval investment: $95.24
portfolio value: $5749930.83
portfolio return: 23858.05%
--------------------------------
annual investment: 24000
interval: W
interval investment: $461.54
portfolio value: $5753303.18
portfolio return: 23872.1%
--------------------------------
annual investment: 24000
interval: 2W
interval investment: $923.08
portfolio value: $5745905.64
portfolio return: 23841.27%
--------------------------------
annual investment: 24000
interval: M
interval investment: $2000.0
portfolio value: $5776872.56
portfolio return: 23970.3%
